In [1]:
from bddl.knowledge_base import *

Loading BDDL knowledge base... This may take a few seconds.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Cem\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Complained object azqqwx does not exist in the database. Skipping.
Complained object ntmrkt does not exist in the database. Skipping.


In [29]:
# Load the existing complaints
import glob, pathlib, json
def load_complaints():
    jsons = {}
    complaint_glob = r"D:/ig_pipeline/cad/*/*/complaints.json"
    for fn in glob.glob(complaint_glob):
        fn = pathlib.Path(fn)
        with open(fn, "r") as f:
            jsons["/".join(fn.parts[-3:-1])] = json.load(f)
    return jsons

def add_complaint(provider, obj, typ, complaint, additional_info):
    complaints = load_complaints()
    max_id = max(c["id"] for cs in complaints.values() for c in cs)
    new_complaint = {
        "id": max_id + 1,
        "object": obj,
        "type": typ,
        "complaint": complaint,
        "additional_info": additional_info,
        "new": True,
        "processed": False,
    }
    complaints[provider].append(new_complaint)
    complaint_path = pathlib.Path(f"D:/ig_pipeline/cad/{provider}/complaints.json")
    complaint_path.parent.mkdir(parents=True, exist_ok=True)
    with open(complaint_path, "w") as f:
        json.dump(complaints[provider], f, indent=2)

In [6]:
# Load object lists also
object_lists = {}
object_list_glob = r"D:/ig_pipeline/cad/*/*/artifacts/object_list.json"
for fn in glob.glob(object_list_glob):
    fn = pathlib.Path(fn)
    with open(fn, "r") as f:
        object_lists["/".join(fn.parts[-3:-1])] = json.load(f)

In [9]:
# Find objects that have glass parts
import sys
sys.path.append(r"D:\ig_pipeline")
from b1k_pipeline.utils import parse_name
glass_part_objects = set()
for objlist in object_lists.values():
    for part in objlist["meshes"]:
        if "Tglass" not in part:
            continue
        pn = parse_name(part)
        glass_part_objects.add(pn.group("model_id"))

In [30]:
windows = set(o.name for o in Synset.get("window.n.01").matching_objects)
windows_without_glass = windows - glass_part_objects
for obj_id in windows_without_glass:
    o = Object.get(obj_id)
    provider = o.provider
    add_complaint(provider, str(o), "appearance", "Automatic complaint: window object has no glass part. Investigate & fix if necessary.", "Some legacy glass objects don't have a glass pane at all which needs to be added manually so that stuff can't go through the window. Other windows like in the gates bedroom are actually not glass in which case this complaint can be removed.")

In [31]:
pictures = set(o.name for o in Category.get("picture").objects)
pictures_with_glass = pictures & glass_part_objects
print(len(pictures_with_glass), "pictures with glass")
for obj_id in pictures_with_glass:
    o = Object.get(obj_id)
    provider = o.provider
    add_complaint(provider, str(o), "appearance", "Automatic complaint: picture object has glass part. It's not necessary semantically and causes performance issues. Remove it.", "The glass in a picture object does not serve any practical purpose in simulation, it just adds some glare to the picture at the cost of compute. We want to avoid this.")

15 pictures with glass
